### 기상청 날씨 데이터 파싱
- BeautifulSoup 라이브러리의 find(),find_all() 함수
- 파싱한 데이터를 list, dict에 저장
- json 파일로 저장하고 읽기

In [6]:
import requests
from bs4 import BeautifulSoup

url = "http://www.kma.go.kr/weather/forecast/mid-term-rss3.jsp"
# GET 요청
res = requests.get(url)
# print(res.status_code)
html = res.text
soup = BeautifulSoup(html, 'html.parser')
#print(soup)

# find()함수 사용 - 1개의 DATA 찾기
title = soup.find('title') # title 태그 데이터 
#print(type(title),title, title.string, title.text)

# find_all()함수 사용 - 모든 DATA 찾기
title_all = soup.find_all('title') # title 태그 데이터 
print(type(title_all), title_all)
# 찾은 데이터 중 텍스트만 출력
for title_tag in title_all:
    print(title_tag.string)

for header in soup.find_all('header'):
    #print(header)
    title = header.find('title').text # title 태그 내용
    tm = header.find('tm').text #tm 태그 내용
    wf = header.find('wf').text #wf 태그 내용
    print(title, tm, wf)


<class 'bs4.element.ResultSet'> [<title>기상청 육상 중기예보</title>, <title>전국 육상 중기예보 - 2021년 01월 10일 (일)요일 06:00 발표</title>, <title>전국 육상중기예보</title>]
기상청 육상 중기예보
전국 육상 중기예보 - 2021년 01월 10일 (일)요일 06:00 발표
전국 육상중기예보
전국 육상중기예보 202101100600 ○ (강수) 13일(수) 강원영서에 눈이 오겠고, 16일(토)은 수도권과 강원영서, 충청권, 전북에 비 또는 눈이 오겠습니다.<br />○ (기온) 13일(수)~15일(금) 아침 기온은 -8~8도, 낮 기온은 4~14도로 어제(아침 기온 -22~-10도, 낮 기온 -8~0도)보다 높겠습니다.<br />          16일(토)~20일(수)은 13일(수)~15일(금)보다 기온이 낮아져 아침 기온은 -11~5도, 낮 기온은 -1~9도가 되겠습니다.<br />○ (해상) 16일(토)~17일(일) 동해상에서 바람이 강하게 불고, 물결이 2.0~4.0m로 높게 일겠습니다.


In [7]:
# location 태그 개수
print(len(soup.find_all('location',attrs={'wl_ver':'3'}))) # <location wl_ver =3>
location_one = soup.find('location',attrs={'wl_ver':'3'}) 
#print(location_one)
province = location_one.find('province').text
city = location_one.find('city').text
print(province, city)
print(len(location_one.find_all('data')))
for data in location_one.find_all('data'):
    #print(data)
    mode = data.find('mode').text
    tmef = data.find('tmef').text
    wf = data.find('wf').text
    tmn = data.find('tmn').text
    tmx = data.find('tmx').text
    print(mode, tmef, wf, tmn, tmx)

41
서울ㆍ인천ㆍ경기도 서울
13
A02 2021-01-13 00:00 맑음 -2 6
A02 2021-01-13 12:00 맑음 -2 6
A02 2021-01-14 00:00 맑음 -2 7
A02 2021-01-14 12:00 맑음 -2 7
A02 2021-01-15 00:00 구름많음 2 9
A02 2021-01-15 12:00 흐림 2 9
A02 2021-01-16 00:00 흐리고 비/눈 -3 0
A02 2021-01-16 12:00 맑음 -3 0
A02 2021-01-17 00:00 맑음 -8 -1
A02 2021-01-17 12:00 맑음 -8 -1
A01 2021-01-18 00:00 맑음 -8 0
A01 2021-01-19 00:00 맑음 -7 2
A01 2021-01-20 00:00 구름많음 -4 3


In [8]:
# Dictionary에 파싱한 데이터 저장
location_dict = {}
location_one = soup.find('location',attrs={'wl_ver':'3'}) # 그 중 한개만 출력
#print(location_one)
location_dict['province'] = location_one.find('province').text
location_dict['city'] = location_one.find('city').text
print(location_dict) # {'province': '서울ㆍ인천ㆍ경기도', 'city': '서울'}

data_list = []
for data in location_one.find_all('data'):
    data_dict = {}
    data_dict['mode'] = data.find('mode').text
    data_dict['tmef'] = data.find('tmef').text
    data_dict['wf'] = data.find('wf').text
    data_dict['tmn'] = data.find('tmn').text
    data_dict['tmx'] = data.find('tmx').text
    data_list.append(data_dict)

location_dict['datas'] = data_list
print(location_dict)

{'province': '서울ㆍ인천ㆍ경기도', 'city': '서울'}
{'province': '서울ㆍ인천ㆍ경기도', 'city': '서울', 'datas': [{'mode': 'A02', 'tmef': '2021-01-13 00:00', 'wf': '맑음', 'tmn': '-2', 'tmx': '6'}, {'mode': 'A02', 'tmef': '2021-01-13 12:00', 'wf': '맑음', 'tmn': '-2', 'tmx': '6'}, {'mode': 'A02', 'tmef': '2021-01-14 00:00', 'wf': '맑음', 'tmn': '-2', 'tmx': '7'}, {'mode': 'A02', 'tmef': '2021-01-14 12:00', 'wf': '맑음', 'tmn': '-2', 'tmx': '7'}, {'mode': 'A02', 'tmef': '2021-01-15 00:00', 'wf': '구름많음', 'tmn': '2', 'tmx': '9'}, {'mode': 'A02', 'tmef': '2021-01-15 12:00', 'wf': '흐림', 'tmn': '2', 'tmx': '9'}, {'mode': 'A02', 'tmef': '2021-01-16 00:00', 'wf': '흐리고 비/눈', 'tmn': '-3', 'tmx': '0'}, {'mode': 'A02', 'tmef': '2021-01-16 12:00', 'wf': '맑음', 'tmn': '-3', 'tmx': '0'}, {'mode': 'A02', 'tmef': '2021-01-17 00:00', 'wf': '맑음', 'tmn': '-8', 'tmx': '-1'}, {'mode': 'A02', 'tmef': '2021-01-17 12:00', 'wf': '맑음', 'tmn': '-8', 'tmx': '-1'}, {'mode': 'A01', 'tmef': '2021-01-18 00:00', 'wf': '맑음', 'tmn': '-8', 'tmx': '0'}, {

### 전국의 날씨정보를 dict와 list에 저장하기

In [9]:
import requests
from bs4 import BeautifulSoup

url = "http://www.kma.go.kr/weather/forecast/mid-term-rss3.jsp"
# GET 요청
res = requests.get(url)
#print(res.status_code)
html = res.text
soup = BeautifulSoup(html, 'html.parser')

# 41개의 lcoation 정보 저장할 리스트 선언
location_list = []
location_all_tag = soup.find_all('location', attrs = {'wl_ver':'3'})
#print(len(location_all_tag))
for location_one in location_all_tag:
    # 1개의 location 정보를 저장할 dict 선언
    location_dict = {}
    location_dict['province'] = location_one.find('province').text
    location_dict['city'] = location_one.find('city').text
    
    # 1개의 location에 포함된 13개의 data를 저장할 list 선언
    data_list = []
    for data in location_one.find_all('data'):
        # 1개의 data 저장할 dict 선언
        data_dict = {}
        # <data><mode>
        data_dict['mode'] = data.find('mode').text
        data_dict['tmef'] = data.find('tmef').text
        data_dict['wf'] = data.find('wf').text
        data_dict['tmn'] = data.find('tmn').text
        data_dict['tmx'] = data.find('tmx').text
        data_list.append(data_dict)
    location_dict['datas']=data_list
    location_list.append(location_dict)
    
print(location_list)

[{'province': '서울ㆍ인천ㆍ경기도', 'city': '서울', 'datas': [{'mode': 'A02', 'tmef': '2021-01-13 00:00', 'wf': '맑음', 'tmn': '-2', 'tmx': '6'}, {'mode': 'A02', 'tmef': '2021-01-13 12:00', 'wf': '맑음', 'tmn': '-2', 'tmx': '6'}, {'mode': 'A02', 'tmef': '2021-01-14 00:00', 'wf': '맑음', 'tmn': '-2', 'tmx': '7'}, {'mode': 'A02', 'tmef': '2021-01-14 12:00', 'wf': '맑음', 'tmn': '-2', 'tmx': '7'}, {'mode': 'A02', 'tmef': '2021-01-15 00:00', 'wf': '구름많음', 'tmn': '2', 'tmx': '9'}, {'mode': 'A02', 'tmef': '2021-01-15 12:00', 'wf': '흐림', 'tmn': '2', 'tmx': '9'}, {'mode': 'A02', 'tmef': '2021-01-16 00:00', 'wf': '흐리고 비/눈', 'tmn': '-3', 'tmx': '0'}, {'mode': 'A02', 'tmef': '2021-01-16 12:00', 'wf': '맑음', 'tmn': '-3', 'tmx': '0'}, {'mode': 'A02', 'tmef': '2021-01-17 00:00', 'wf': '맑음', 'tmn': '-8', 'tmx': '-1'}, {'mode': 'A02', 'tmef': '2021-01-17 12:00', 'wf': '맑음', 'tmn': '-8', 'tmx': '-1'}, {'mode': 'A01', 'tmef': '2021-01-18 00:00', 'wf': '맑음', 'tmn': '-8', 'tmx': '0'}, {'mode': 'A01', 'tmef': '2021-01-19 00:0

### 전국의 날씨정보를 json 파일로 저장 및 읽기

In [10]:
import json

# json 파일 생성
with open('data/weather_20210110.json','w') as file:
    json.dump(location_list, file)

In [11]:
import json

# json 읽기
with open('data/weather_20210110.json') as file:
    contents = file.read()
    weather_json = json.loads(contents)
    
print(weather_json)

[{'province': '서울ㆍ인천ㆍ경기도', 'city': '서울', 'datas': [{'mode': 'A02', 'tmef': '2021-01-13 00:00', 'wf': '맑음', 'tmn': '-2', 'tmx': '6'}, {'mode': 'A02', 'tmef': '2021-01-13 12:00', 'wf': '맑음', 'tmn': '-2', 'tmx': '6'}, {'mode': 'A02', 'tmef': '2021-01-14 00:00', 'wf': '맑음', 'tmn': '-2', 'tmx': '7'}, {'mode': 'A02', 'tmef': '2021-01-14 12:00', 'wf': '맑음', 'tmn': '-2', 'tmx': '7'}, {'mode': 'A02', 'tmef': '2021-01-15 00:00', 'wf': '구름많음', 'tmn': '2', 'tmx': '9'}, {'mode': 'A02', 'tmef': '2021-01-15 12:00', 'wf': '흐림', 'tmn': '2', 'tmx': '9'}, {'mode': 'A02', 'tmef': '2021-01-16 00:00', 'wf': '흐리고 비/눈', 'tmn': '-3', 'tmx': '0'}, {'mode': 'A02', 'tmef': '2021-01-16 12:00', 'wf': '맑음', 'tmn': '-3', 'tmx': '0'}, {'mode': 'A02', 'tmef': '2021-01-17 00:00', 'wf': '맑음', 'tmn': '-8', 'tmx': '-1'}, {'mode': 'A02', 'tmef': '2021-01-17 12:00', 'wf': '맑음', 'tmn': '-8', 'tmx': '-1'}, {'mode': 'A01', 'tmef': '2021-01-18 00:00', 'wf': '맑음', 'tmn': '-8', 'tmx': '0'}, {'mode': 'A01', 'tmef': '2021-01-19 00:0